# Hierarchical Mini Blog Post Writer Crew - Research with Human Input - CrewAI.ipynb


https://pypi.org/project/crewai/

In [ ]:
!pip -q install crewai duckduckgo-search unstructured
!pip -q install 'crewai[tools]' decouple


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.0/51.0 kB 543.9 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 810.5/810.5 kB 24.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 262.9/262.9 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.1/60.1 kB 1.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.1/106.1 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.7/5.7 MB 18.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.8/144.8 kB 1.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 421.5/421.5 kB 27.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 50.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.0/8.0 MB 38.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 2.3 MB/

In [ ]:
!pip show crewai

Name: crewai
Version: 0.22.5
Summary: Cutting-edge framework for orchestrating role-playing, autonomous AI agents. By fostering collaborative intelligence, CrewAI empowers agents to work together seamlessly, tackling complex tasks.
Home-page: 
Author: Joao Moura
Author-email: joao@crewai.com
License: 
Location: /usr/local/lib/python3.10/dist-packages
Requires: click, instructor, langchain, langchain-openai, openai, opentelemetry-api, opentelemetry-exporter-otlp-proto-http, opentelemetry-sdk, pydantic, python-dotenv, regex
Required-by: 


In [ ]:
import os
from google.colab import userdata

os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')

## Utilities

In [ ]:
import json  # Import the JSON module to parse JSON strings
from langchain_core.agents import AgentFinish

agent_finishes  = []

import json
from typing import Union, List, Tuple, Dict
from langchain.schema import AgentFinish


call_number = 0

def print_agent_output(agent_output: Union[str, List[Tuple[Dict, str]], AgentFinish], agent_name: str = 'Generic call'):
    global call_number  # Declare call_number as a global variable
    call_number += 1
    with open("crew_callback_logs.txt", "a") as log_file:
        # Try to parse the output if it is a JSON string
        if isinstance(agent_output, str):
            try:
                agent_output = json.loads(agent_output)  # Attempt to parse the JSON string
            except json.JSONDecodeError:
                pass  # If there's an error, leave agent_output as is

        # Check if the output is a list of tuples as in the first case
        if isinstance(agent_output, list) and all(isinstance(item, tuple) for item in agent_output):
            print(f"-{call_number}----Dict------------------------------------------", file=log_file)
            for action, description in agent_output:
                # Print attributes based on assumed structure
                print(f"Agent Name: {agent_name}", file=log_file)
                print(f"Tool used: {getattr(action, 'tool', 'Unknown')}", file=log_file)
                print(f"Tool input: {getattr(action, 'tool_input', 'Unknown')}", file=log_file)
                print(f"Action log: {getattr(action, 'log', 'Unknown')}", file=log_file)
                print(f"Description: {description}", file=log_file)
                print("--------------------------------------------------", file=log_file)

        # Check if the output is a dictionary as in the second case
        elif isinstance(agent_output, AgentFinish):
            print(f"-{call_number}----AgentFinish---------------------------------------", file=log_file)
            print(f"Agent Name: {agent_name}", file=log_file)
            agent_finishes.append(agent_output)
            # Extracting 'output' and 'log' from the nested 'return_values' if they exist
            output = agent_output.return_values
            # log = agent_output.get('log', 'No log available')
            print(f"AgentFinish Output: {output['output']}", file=log_file)
            # print(f"Log: {log}", file=log_file)
            # print(f"AgentFinish: {agent_output}", file=log_file)
            print("--------------------------------------------------", file=log_file)

        # Handle unexpected formats
        else:
            # If the format is unknown, print out the input directly
            print(f"-{call_number}-Unknown format of agent_output:", file=log_file)
            print(type(agent_output), file=log_file)
            print(agent_output, file=log_file)



In [ ]:
from crewai import Crew, Agent, Task, Process
from langchain_community.tools import DuckDuckGoSearchRun

## Tools

In [ ]:
from langchain_openai import ChatOpenAI

# Initialize the OpenAI GPT-4 language model
OpenAIGPT4TURBO = ChatOpenAI(
    model="gpt-4-turbo-preview"
)

In [ ]:
from datetime import datetime
from random import randint
from langchain.tools import tool

@tool("save_content")
def save_content(task_output):
    """Useful to save content to a markdown file"""
    print('in the save markdown tool')
    # Get today's date in the format YYYY-MM-DD
    today_date = datetime.now().strftime('%Y-%m-%d')
    # Set the filename with today's date
    filename = f"{today_date}_{randint(0,100)}.md"
    # Write the task output to the markdown file
    with open(filename, 'w') as file:
        file.write(task_output)
        # file.write(task_output.result)

    print(f"Blog post saved as {filename}")

    return f"Blog post saved as {filename}, please tell the user we are finished"

In [ ]:
import json
import os

import requests
from langchain.tools import tool

import os
from crewai import Agent, Task, Crew, Process
from langchain_community.tools import DuckDuckGoSearchRun
from langchain.agents import load_tools

search_tool = DuckDuckGoSearchRun()

# Loading Human Tools
human_tools = load_tools(["human"])

## Agents

In [ ]:
from crewai import Agent

# Define your agents with roles and goals
topic_getter = Agent(
    role='A Senior customer communicator',
    goal='consult with the human customer to get the topic and areas of interest for doing the research',
    backstory="""As a top customer communicator at a renowned technology you have honed your skills
    in consulting with a customer to understand their needs and goals for what research is needed.""",
    verbose=True,
    allow_delegation=False,
    llm=OpenAIGPT4TURBO,
    max_iter=5,
    memory=True,
    step_callback=lambda x: print_agent_output(x,"Senior Customer Agent"),
    tools= human_tools, # Passing human tools to the agent,
)


info_getter = Agent(
    role='AI Research Specialist',
    goal='Leverage advanced search techniques to surface the most relevant, credible, and impactful information on AI and data science breakthroughs',
    backstory="""As a top AI Research Specialist at a renowned technology
    research institute, you have honed your skills in crafting sophisticated
    search queries, filtering information from trusted sources, and synthesizing
    key insights. You have the ability to take a topic suggested by a human and
    rewrite multiple searches for that topic to get the best results overall.

    Your extensive knowledge of AI and data science, combined
    with your mastery of machine learning and Large Language models, allows you
    to unearth groundbreaking research that others often overlook. You excel
    at critically evaluating the credibility and potential
    impact of new developments, enabling you to curate a focused feed of the most
    significant advances. Your talent for clear and concise summarization helps
    you distill complex technical concepts into easily digestible executive
    briefings and reports. With a track record of consistently identifying
    paradigm-shifting innovations before they hit the mainstream, you have become
    the go-to expert for keeping your organization at the forefront of the AI revolution.""",
    verbose=True,
    allow_delegation=False,
    llm=OpenAIGPT4TURBO,
    max_iter=5,
    memory=True,
    step_callback=lambda x: print_agent_output(x,"Senior Research Analyst Agent"),
    tools=[search_tool]
)


writer = Agent(
    role='Tech Content Writer and rewriter',
    goal='Generate compelling content via first drafts and subsequent polishing to get a final product. ',
    backstory="""As a renowned Tech Content Strategist, you have a gift for transforming complex technical
    concepts into captivating and easily digestible articles. Your extensive knowledge of the tech
    industry allows you to identify the most compelling angles and craft narratives that resonate
    with a wide audience.

    Your writing prowess extends beyond simply conveying information; you have a knack for restructuring
    and formatting content to enhance readability and engagement. Whether it's breaking down intricate
    ideas into clear, concise paragraphs or organizing key points into visually appealing lists,
    your articles are a masterclass in effective communication.

    Some of your signature writing techniques include:

    Utilizing subheadings and bullet points to break up long passages and improve scannability

    Employing analogies and real-world examples to simplify complex technical concepts

    Incorporating visuals, such as diagrams and infographics, to supplement the written content

    Varying sentence structure and length to maintain a dynamic flow throughout the article

    Crafting compelling introductions and conclusions that leave a lasting impact on readers

    Your ability to rewrite and polish rough drafts into publishable masterpieces is unparalleled.
    You have a meticulous eye for detail and a commitment to delivering content that not only informs
    but also engages and inspires. With your expertise, even the most technical and dry subject matter
    can be transformed into a riveting read.""",
    llm=OpenAIGPT4TURBO,
    verbose=True,
    # allow_delegation=True,
    max_iter=5,
    memory=True,
    step_callback=lambda x: print_agent_output(x,"Content Writer Agent"),
    allow_delegation=True,
    # tools=[search_tool], # Passing human tools to the agent,
)

archiver = Agent(
    role='File Archiver',
    goal='Take in information and write it to a Markdown file',
    backstory="""You are a efficient and simple agent that gets data and saves it to a markdown file. in a quick and efficient manner""",
    llm=OpenAIGPT4TURBO,
    verbose=True,
    # allow_delegation=True,
    # max_iter=15,
    step_callback=lambda x: print_agent_output(x,"Archiver Agent"),
    # allow_delegation=True,
    tools=[save_content], # Passing human tools to the agent,
)


## Tasks

In [ ]:
from datetime import datetime

# Create tasks for your agents
get_human_topic = Task(
  description=f"""ASK THE HUMAN for the topic and area of interest.

  Compile you results into a clear topic that can be used for doing research going forward""",
  expected_output="""Clearly state the topic that the human wants you to research.\n\n
   eg. HUMAN_TOPIC_FOR_RESEARCH = 'AI_TOPIC' """,
  agent=topic_getter
)

get_source_material = Task(
  description=f"""Take the topic for research and search for it on the internet.
  The current time is {datetime.now()}. Focus on recent events related to the topic for research only.
  Identify key facts and useful information related to the topic of research

  Compile you results into a useful and helpful report for the writer to use to write an article""",
  expected_output='A comprehensive full report on the latest AI advancements in the specified human topic, leave nothing out',
  agent=info_getter
)

write_the_content = Task(
  description="""Using the source material from the research specialist's report,
  develop a nicely formated article that is brief, to the point and highlights the
  most significant information and advancements.
  Your article should be informative yet accessible, catering to a tech-savvy audience.
  Aim for a narrative that captures the essence of these breakthroughs and their
  implications for the future (both for research, but also for industy).
  DON'T overly 'Hype' the topic. Be factual and clear.
  Your final answer MUST be a full article post of at least 3 paragraphs and should contain
  a set of bullet points with the key facts at the end for a summary""",
  expected_output="""A compelling 3 paragraphs article with a set of bullet points with
  the key facts at the end for a summay. This should all be formated as markdown in an easy readable manner""",
  agent=writer
)

saving_the_output = Task(
  description="""Taking the post created by the writer, take this and save it to a markdown file.
  Your final answer MUST be a response must be showing that the file was saved .""",
  expected_output='A saved file name',
  agent=archiver
)




## Crew

In [ ]:
from crewai import Crew, Process

# Instantiate your crew with a sequential process
crew = Crew(
    agents=[topic_getter,
            info_getter,
            writer,
            archiver],
    tasks=[get_human_topic,
           get_source_material,
           write_the_content,
           saving_the_output],
    verbose=2,
    process=Process.hierarchical,
    full_output=True,
    share_crew=False,
    manager_llm=OpenAIGPT4TURBO,
    max_iter=15,
    step_callback=lambda x: print_agent_output(x,"MasterCrew Agent")
)

In [ ]:
# Kick off the crew's work
results = crew.kickoff()

# Print the results
print("Crew Work Results:")
print(results)

 [DEBUG]: Working Agent: Crew Manager
 [INFO]: Starting Task: ASK THE HUMAN for the topic and area of interest.

  Compile you results into a clear topic that can be used for doing research going forward


> Entering new CrewAgentExecutor chain...
Thought: Before I can compile results into a clear topic for research as requested, I need to know the human's topic and area of interest. Given my role and the tools at my disposal, I should start by asking the Senior customer communicator, as they are likely skilled in identifying and clarifying such inquiries from humans.

Action: Ask question to co-worker

Action Input: {"coworker": "A Senior customer communicator", "question": "What is the current topic and area of interest that the human wants us to research?", "context": "We have been tasked with identifying a specific topic and area of interest for research purposes. This is crucial for directing our efforts and resources efficiently. The information on the human's preferences or requ

In [ ]:
# prompt: import ipython display and markdown and then use markdown to print the results variable

from IPython.display import display, Markdown

# Convert the results variable to a Markdown object
markdown_results = Markdown(str(results['final_output']))

# Display the Markdown object
display(markdown_results)


The content was successfully saved in a file named "2024-04-01_57.md". The file contains detailed information on Jamba's innovative AI model developed by AI21 Labs, discussing its hybrid architecture, applications in various industries, recent advancements, positive reception in the tech community, and its implications for the future of AI. The markdown formatting, including headers, bold text for key points, and bullet points for the summary section, has been preserved to ensure clarity and readability.

In [ ]:
! cat /content/2024-04-01_57.md

# Jamba: A Revolutionary AI Model by AI21 Labs

Jamba, an innovative AI model developed by AI21 Labs, represents a significant leap in the field of artificial intelligence. With its unique hybrid architecture that combines Mamba SSM technology with Transformer architecture, Jamba stands out for its versatility and efficiency.

## Key Features
- **Hybrid Architecture**: Combining Mamba SSM technology with Transformer architecture for enhanced performance.
- **Diverse Applications**: Useful across various industries including healthcare, customer service, and finance.
- **Continuous Improvement**: Jamba is constantly being improved upon, with recent advancements in natural language processing.
- **Positive Industry Reception**: The tech community has welcomed Jamba with open arms, recognizing its potential to shape the future of AI.

## Future Implications
The introduction of Jamba into the market signifies a promising future for AI applications. Its innovative approach and the positive 

In [ ]:
!cat /content/crew_callback_logs.txt

-28----Dict------------------------------------------
Agent Name: Senior Customer Agent
Tool used: human
Tool input: {"question": "Could you please specify the topic and area of interest you want us to research?"}

Action log: Thought: Before proceeding with any research or action, I need to understand the specific topic and area of interest that the human wants us to research. This will guide our efforts effectively. To do this, I must directly ask the human for this information.

Action: human

Action Input: {"question": "Could you please specify the topic and area of interest you want us to research?"}

Description: Jamba: AI21's SSM-Transformer Model
--------------------------------------------------
-29----Dict------------------------------------------
Agent Name: Senior Customer Agent
Tool used: human
Tool input: {"question": "I apologize for any confusion. Could you clarify the specific topic and area of interest you are looking for us to conduct research on?"}

Action log: Thou

In [ ]:
from IPython.display import display, Markdown

# Load the Markdown file
with open("/content/2024-04-01_57.md", "r") as file:
    markdown_content = file.read()

# Create a Markdown object from the loaded content
markdown_results = Markdown(markdown_content)

# Display the Markdown object
display(markdown_results)


# Jamba: A Revolutionary AI Model by AI21 Labs

Jamba, an innovative AI model developed by AI21 Labs, represents a significant leap in the field of artificial intelligence. With its unique hybrid architecture that combines Mamba SSM technology with Transformer architecture, Jamba stands out for its versatility and efficiency.

## Key Features
- **Hybrid Architecture**: Combining Mamba SSM technology with Transformer architecture for enhanced performance.
- **Diverse Applications**: Useful across various industries including healthcare, customer service, and finance.
- **Continuous Improvement**: Jamba is constantly being improved upon, with recent advancements in natural language processing.
- **Positive Industry Reception**: The tech community has welcomed Jamba with open arms, recognizing its potential to shape the future of AI.

## Future Implications
The introduction of Jamba into the market signifies a promising future for AI applications. Its innovative approach and the positive reception it has garnered suggest that Jamba will play a pivotal role in advancing AI technology.

Jamba's development reflects AI21 Labs' commitment to pushing the boundaries of AI research and application, making it a model worth watching in the years to come.